# Analyzing borrowers’ risk of defaulting

Your project is to prepare a report for a bank’s loan division. You’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Your report will be considered when building a **credit scoring** of a potential customer. A ** credit scoring ** is used to evaluate the ability of a potential borrower to repay their loan.

## Open the data file and have a look at the general information. 

In [ ]:
import pandas as pd #first read data
df = pd.read_csv('/datasets/credit_scoring_eng.csv')
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Conclusion

The data is readable via pandas:

1. There are 12 columns: children,	days_employed,	dob_years,	education,	education_id,	family_status,	family_status_id,	gender	income_type	debt,	total_income,	purpose

2. One row represents a bank loan division customer's creditworthiness based on the columns.

3. We have the information of 21,525 customers (each row representing a single customer)

4. There are no user_id values; the unique identifier is simply the index number

5. By looking at values in total_income and days_employed, the data would look better if they were expressed as int() rather than float

6. Using df.info() there appears to be no missing values (null), however, it will still be investigated in the data preprocessing

## Data preprocessing

### Processing missing values

In [ ]:
#First, find any 'None' values in the dataset
none_values = df[df == 'None'].sum()
none_values

#Results: confirmed no 'None' type missing values

/opt/conda/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


children            0.0
days_employed       0.0
dob_years           0.0
education           0.0
education_id        0.0
family_status       0.0
family_status_id    0.0
gender              0.0
income_type         0.0
debt                0.0
total_income        0.0
purpose             0.0
dtype: float64

In [ ]:
#First, find out how many missing values there are 
df.isnull().sum()
#Results: days_employed and total_income both have 2174 missing values
#Does this necessarily mean that these are all unemployed persons?

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [ ]:
#Next, calculate percentage to view how the missing values impact the data
df.isnull().sum()/len(df)
#The missing data effects more than 5% of the data, therefore, it should be filled in with
#a representative value -- the mean

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

In [ ]:
df['income_type'].value_counts()
#only two customers have unemployment status, therefore, the rows with missing values in 
#days_employed and total_income are not necessarily unemployed.
#These values must be filled in with the respective means

employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

In [ ]:
df['children'].value_counts()
#There are -1 and 20 values for children -- it is most likely a typo for 1 and 2

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [ ]:
df.loc[df['children'] == -1, 'children'] = 1
df['children'].value_counts()
#Next, fix 20

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

In [ ]:
df.loc[df['children'] == 20, 'children'] = 2
df['children'].value_counts()
#Erroneous values have been changed

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [ ]:
df['days_employed'] = df['days_employed'].abs()
days_employed_mean = df['days_employed'].mean().astype('int')

#replace missing values with 'unknown' -- will be replaced by mean
df['days_employed'] = df['days_employed'].fillna(days_employed_mean) 
df['days_employed'].isnull().sum() #checking if missing values were filled in
#it works!
#Now, mean vs. median for total_income?

0

In [ ]:
total_income_median = df['total_income'].median().astype('int')
total_income_mean = df['total_income'].mean().astype('int')
print(total_income_median)
print(total_income_mean)
#The mean and median are not too different from eachother, therefore the mean is used by default
#because it is the most familiar metric for the client

25024
26787


In [ ]:
total_income_mean = df['total_income'].mean().astype('int')
df['total_income'] = df['total_income'].fillna(total_income_mean)
df['total_income'].head(15)
#there are no more missing values; all are now replaced by means

0     40620.102
1     17932.802
2     23341.752
3     42820.568
4     25378.572
5     40922.170
6     38484.156
7     21731.829
8     15337.093
9     23108.150
10    18230.959
11    12331.077
12    26787.000
13    20873.317
14    26420.466
Name: total_income, dtype: float64

### Conclusion

1) 'days_employed' and 'total_income' had the same amount of missing values as eachother. This could be explained by customer who are not technically 'employed' but rather as a contracted position ie. 1099 contracter. These worker's total income may vary, explaining why their total_income was also missing.
   
2) 'None' type values were confirmed non-existant in the data.

3) Missing values were replaced by representative values for 'days_employed' and 'total_income'

### Data type replacement

In [ ]:
#Expressing 'total_income' values as an integer
df['total_income'] = df['total_income'].fillna(total_income_mean).astype('int')
df['total_income'].head(15)
#using the mean to fill in missing values will help preserve data rather than deleting rows with missing values

0     40620
1     17932
2     23341
3     42820
4     25378
5     40922
6     38484
7     21731
8     15337
9     23108
10    18230
11    12331
12    26787
13    20873
14    26420
Name: total_income, dtype: int64

### Conclusion

Missing values for days_employed and total_income were filled in using representative values (mean). days_employed and total_income are now integer data types, and days_employed values are now positive.

astype() must be used because int() is used only for single values while astype() can be used for a series



### Processing duplicates

Next, must find duplicate entries for categorical variable column, such as education, purpose,
income_type, and family_status.
Since a duplicate value in one of these columns does not mean there is a duplicate customer, 
they must simply be re-written so there is only one of each value; no rows need to be deleted starting with 'education' 

In [ ]:
df['education'].value_counts()
#Used value_counts() to identify the number of duplicate values for each unique value

secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
Graduate Degree            1
GRADUATE DEGREE            1
Name: education, dtype: int64

In [ ]:
#Education needs to be processed with case sensitive methods:
df['education'] = df['education'].str.lower()
df['education'].value_counts()
#a new table df['education_low'] compiles all 'education' values in lowercase for processing
#.value_counts() shows that each value has succesfully been converted into only one version
#Next, family_status

secondary education    15233
bachelor's degree       5260
some college             744
primary education        282
graduate degree            6
Name: education, dtype: int64

In [ ]:
df['family_status'].value_counts()
#'family_status' does not have duplicate values
#Next, 'income_type'

married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64

In [ ]:
df['income_type'].value_counts()
#'income_type' also has no duplicate values
#Next, purpose

employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

In [ ]:
df['purpose'].value_counts()
#Results: no uppercase values
#The outputs shows there are duplicate entries for varies purposes i.e 'wedding ceremony',
#'having a wedding', and 'to have a wedding' are all the same

wedding ceremony                            797
having a wedding                            777
to have a wedding                           774
real estate transactions                    676
buy commercial real estate                  664
buying property for renting out             653
housing transactions                        653
transactions with commercial real estate    651
purchase of the house                       647
housing                                     647
purchase of the house for my family         641
construction of own property                635
property                                    634
transactions with my real estate            630
building a real estate                      626
buy real estate                             624
purchase of my own house                    620
building a property                         620
housing renovation                          612
buy residential real estate                 607
buying my own car                       

### Conclusion

Duplicate entries were processed for categorical variable column, such as education and purpose. Family_status and income_type did not have any duplicate values.

Education was processed via str.lower() which allows the duplicate values to be counted for as one value

Purpose was processed differently; there were many ways to say a person was buying a house. Therefore, normalization via stemming and lemmatization to count the lemmas for each word was performed, which resulted in a list of lemmas the client can use to observe popular key words in regards to the purpose of obtaining a loan.

The values in the purpose column are still ambiguous (ie. housing, house, estate are still separate values but should be counted the same). Further categorization is still required. 

### Categorizing Data

In [ ]:
#First, normalize the data: combine all purpose values into one string
allwords= ' '.join(df['purpose'])
allwords

'purchase of the house car purchase purchase of the house supplementary education to have a wedding purchase of the house housing transactions education having a wedding purchase of the house for my family buy real estate buy commercial real estate to have a wedding car purchase buy residential real estate construction of own property property building a property buying a second-hand car buying my own car property car purchase buying a second-hand car to have a wedding transactions with commercial real estate buy real estate education construction of own property construction of own property building a real estate transactions with commercial real estate housing having a wedding purchase of the house transactions with my real estate having a wedding cars car purchase education to become educated buy real estate second-hand car purchase getting an education car purchase buying my own car to become educated second-hand car purchase having a wedding construction of own property car cars w

In [ ]:
#Next step in normalization: lemmatization, so it does not confuse different versions of the
#same word
import nltk
from nltk.stem import WordNetLemmatizer
wordnet_lemma = WordNetLemmatizer()

#change allwords to a list of words: lemmas
allwords = nltk.word_tokenize(allwords)
lemmas = [wordnet_lemma.lemmatize(w, pos='n') for w in allwords]
lemmas

['purchase',
 'of',
 'the',
 'house',
 'car',
 'purchase',
 'purchase',
 'of',
 'the',
 'house',
 'supplementary',
 'education',
 'to',
 'have',
 'a',
 'wedding',
 'purchase',
 'of',
 'the',
 'house',
 'housing',
 'transaction',
 'education',
 'having',
 'a',
 'wedding',
 'purchase',
 'of',
 'the',
 'house',
 'for',
 'my',
 'family',
 'buy',
 'real',
 'estate',
 'buy',
 'commercial',
 'real',
 'estate',
 'to',
 'have',
 'a',
 'wedding',
 'car',
 'purchase',
 'buy',
 'residential',
 'real',
 'estate',
 'construction',
 'of',
 'own',
 'property',
 'property',
 'building',
 'a',
 'property',
 'buying',
 'a',
 'second-hand',
 'car',
 'buying',
 'my',
 'own',
 'car',
 'property',
 'car',
 'purchase',
 'buying',
 'a',
 'second-hand',
 'car',
 'to',
 'have',
 'a',
 'wedding',
 'transaction',
 'with',
 'commercial',
 'real',
 'estate',
 'buy',
 'real',
 'estate',
 'education',
 'construction',
 'of',
 'own',
 'property',
 'construction',
 'of',
 'own',
 'property',
 'building',
 'a',
 'real',


In [ ]:
#Analyze the frequency of each word
from collections import Counter
Counter(lemmas)
#This will be helpful in choosing key lemmas to filter str values and assign a purpose group

Counter({'purchase': 3314,
         'of': 2998,
         'the': 1288,
         'house': 1908,
         'car': 4315,
         'supplementary': 909,
         'education': 3114,
         'to': 3081,
         'have': 774,
         'a': 5130,
         'wedding': 2348,
         'housing': 1912,
         'transaction': 2610,
         'having': 777,
         'for': 1294,
         'my': 2396,
         'family': 641,
         'buy': 2367,
         'real': 4478,
         'estate': 4478,
         'commercial': 1315,
         'residential': 607,
         'construction': 635,
         'own': 2240,
         'property': 2542,
         'building': 1246,
         'buying': 1637,
         'second-hand': 968,
         'with': 1281,
         'become': 412,
         'educated': 412,
         'getting': 869,
         'an': 443,
         'ceremony': 797,
         'get': 447,
         'higher': 426,
         'profile': 436,
         'university': 949,
         'renting': 653,
         'out': 653,
         'ren

In [ ]:
#Now it is time to assign customers a purpose category by creating a purpose_assign function

df['purpose_lemmas'] = df['purpose'].str.split().apply(lambda x: [wordnet_lemma.lemmatize(y, pos='n') for y in x])
#split turns str values in 'purpose' into a list that can be used to use the apply method -- apply is useful for applyin
#the lemmatization for each row
def purpose_assign(purpose):
        if 'house' in purpose or 'housing' in purpose or 'estate' in purpose or 'property' in purpose or 'building' in purpose or 'renting' in purpose or 'renovation' in purpose:
            return 'property'
        elif 'education' in purpose or 'university' in purpose or 'educated' in purpose:
            return 'education'
        elif 'wedding' in purpose or 'ceremony'in purpose:
            return 'wedding'
        elif 'car' in purpose:
            return 'car'
        else:
            return 'other'

df['purpose_group'] = df['purpose'].apply(purpose_assign)

df['purpose_group'].value_counts()
#Property is the largest reason people take out a loan!

property     10840
car           4315
education     4022
wedding       2348
Name: purpose_group, dtype: int64

In [ ]:
#Next categorize 'children' column
def children_assign(children):
    if children == 0:
        return 'No children' #Many had no children -- has its' own category
    elif children <= 3: 
        return 'Has children' #Most that had children fell under these numbers -- has own category
    elif children <= 5:
        return 'Has many children' #Fewer fell in this range -- has own category
df['children_assign'] = df['children'].apply(children_assign)
df['children_assign'].value_counts()
#Results: Most customers have no children

No children          14149
Has children          7326
Has many children       50
Name: children_assign, dtype: int64

In [ ]:
#processing missing values
#Next, categorize total_income
def income_assign(income):
    if income <= 30000: #standard concept value for lower income 
        return 'lower income'
    if income <= 50000: #
        return 'low income'
    if income <= 80000:
        return 'medium income'
    if income <= 100000:
        return 'high income'
    else:
        return 'higher income'
df['income_assign'] = df['total_income'].apply(income_assign)
df['income_assign'].value_counts()

lower income     15606
low income        4599
medium income     1098
high income        123
higher income       99
Name: income_assign, dtype: int64

In [ ]:
#Next, categorize 'age'
def age_assign(age):
    if age <=20:
        return '0-20'#some 'customers' may have ages younger including 0 for baby accounts/child savings
    if age <=30:
        return '21-30'
    if age <=40:
        return '31-40'
    if age <=50:
        return '41-50'
    if age <=60:
        return '51-60'
    if age <=70:
        return '61-70'
    else:
        return '70+'
df['age'] = df['dob_years'].apply(age_assign)
df['age'].value_counts()

31-40    5743
41-50    5276
51-60    4542
21-30    3658
61-70    2034
0-20      166
70+       106
Name: age, dtype: int64

In [ ]:
#Finally, start working on the pivot table
df_pivot_main = df.pivot_table(index=['family_status'], columns='children_assign', values='debt', aggfunc=['sum', 'mean'])
df_pivot_main
#Results: df_pivot_main will be used to address questions about family_status and children affecting ability to pay loan

sum                                       mean  \
children_assign   Has children Has many children No children Has children   
family_status                                                               
civil partnership          159                 0         229     0.112367   
divorced                    30                 0          55     0.073171   
married                    412                 3         516     0.085054   
unmarried                   63                 1         210     0.115385   
widow / widower             10                 0          53     0.090090   

                                                 
children_assign   Has many children No children  
family_status                                    
civil partnership          0.000000    0.083212  
divorced                   0.000000    0.070153  
married                    0.083333    0.068800  
unmarried                  0.500000    0.092715  
widow / widower            0.000000    0.062500

In [ ]:
#Next, make pivot table for income and purpose
df_pivot_additional1 = df.pivot_table(index=['income_assign'], columns='purpose_group', values='debt', aggfunc='sum')
df_pivot_additional1 #this pivot table displays debt in relation to income and purpose_group

purpose_group,car,education,property,wedding
income_assign,,,,
high income,3,1,3,1
higher income,0,3,3,0
low income,82,70,162,30
lower income,299,284,573,149
medium income,19,12,41,6


### Conclusion

Summary:

The first step in categorizing the data was to normalize the values of the purpose column via lemmatization. The purpose lemmas list was used to create a function that assigned each customer to a specific purpose group. Children, income, and age were then categorized into specific groups. Categorizing was necessary in order to view the data easier. Two pivot tables, main (addressing the main question) and additional (addresing other questions for income and purpose) were created. One pivot table for all questions resulted in a large 74 row table that was too big for clear results, therefore two were created.

## Answer these questions

- Is there a relation between having kids and repaying a loan on time?

In [ ]:
df_pivot_main

sum                                       mean  \
children_assign   Has children Has many children No children Has children   
family_status                                                               
civil partnership          159                 0         229     0.112367   
divorced                    30                 0          55     0.073171   
married                    412                 3         516     0.085054   
unmarried                   63                 1         210     0.115385   
widow / widower             10                 0          53     0.090090   

                                                 
children_assign   Has many children No children  
family_status                                    
civil partnership          0.000000    0.083212  
divorced                   0.000000    0.070153  
married                    0.083333    0.068800  
unmarried                  0.500000    0.092715  
widow / widower            0.000000    0.062500

According to the df_pivot_main table, kids did have an effect on paying a loan on time. Customers without children had more debt than those customers with children. Keeping in mind, this is a correlation and having children itself does not cause timely loan payment. Customers with children may have a heightened sense of responsibility compared to those without kids. Also, customers with children have added tax benefits, such as increased tax return for claiming dependents, which may help in the loan repayment process.

### Conclusion

- Is there a relation between marital status and repaying a loan on time?

Referring to df_pivot_main table, there was a relation between marital status and loan repayment in that married and civil partnerships had the most debt. This may occur due to increased likeliness of having kids, and those without kids that are married or in a civil partnership may have a lower income or a lesser sense responsibility compared to those with kids

### Conclusion

In [ ]:
df_pivot_additional1

purpose_group,car,education,property,wedding
income_assign,,,,
high income,3,1,3,1
higher income,0,3,3,0
low income,82,70,162,30
lower income,299,284,573,149
medium income,19,12,41,6


    - Is there a relation between income level and repaying a loan on time?

Referring to df_pivot_additional1, lower income had the greatest amount of debt showing that there was a relation between income level and loan repayment

### Conclusion

- How do different loan purposes affect on-time repayment of the loan?

Also referring to df_pivot_additional1, property loans had the highest amount of debt showing that loan purposes do affect loan repayment. This occurs most likely because properties are very expense and the time it takes to repay the loan takes a long time and unexpected changes in income may occur ie. being laid off. 

## General conclusion

1. Cleaned data and created pivot tables to represent relations between marital status, having chlidren, and repaying a loan. 

2. Those with children tend to pay a loan back more than those without children due to various factors that cannot be analyzed with the data provided.

3. Lower income and property loans had the highest relation to untimely loan repayment.

## Project Readiness Checklist

Put 'x' in the completed points. Then press Shift + Enter.

- [x]  file open;
- [x]  file examined;
- [x]  missing values defined;
- [x]  missing values are filled;
- [x ]  an explanation of which missing value types were detected;
- [x]  explanation for the possible causes of missing values;
- [x]  an explanation of how the blanks are filled;
- [x]  replaced the real data type with an integer;
- [x]  an explanation of which method is used to change the data type and why;
- [x]  duplicates deleted;
- [x]  an explanation of which method is used to find and remove duplicates;
- [x]  description of the possible reasons for the appearance of duplicates in the data;
- [x]  data is categorized;
- [x]  an explanation of the principle of data categorization;
- [x]  an answer to the question "Is there a relation between having kids and repaying a loan on time?";
- [x]  an answer to the question " Is there a relation between marital status and repaying a loan on time?";
- [x]   an answer to the question " Is there a relation between income level and repaying a loan on time?";
- [x]  an answer to the question " How do different loan purposes affect on-time repayment of the loan?"
- [x]  conclusions are present on each stage;
- [x]  a general conclusion is made.